# Stable Diffusion XL (SDXL)

In [ ]:
!pip install -q invisible_watermark transformers accelerate safetensors diffusers

# Inference with pipelines
- For base SDXL

In [ ]:
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline, DiffusionPipeline, KDPM2AncestralDiscreteScheduler, StableDiffusionXLPipeline, AutoencoderKL, UNet2DConditionModel, EulerDiscreteScheduler
import gc
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display

In [3]:
model_base = "stabilityai/stable-diffusion-xl-base-1.0"

pipe = StableDiffusionXLPipeline.from_pretrained(
    model_base,
    # vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False, # no watermarker
    )

pipe.safety_checker = None

pipe.to("cuda")

- Refiner of SDXL.

In [ ]:
model_refiner = "stabilityai/stable-diffusion-xl-refiner-1.0"

pipe_refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    model_refiner,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False, # no watermarker
    )

#pipe_refiner.to("cuda")
pipe_refiner.enable_model_cpu_offload()

##  Generation with the base model.

We can provide a single prompt or take two, each with a maximum length of 77 tokens.

In [ ]:
# max 77 tokens in prompt
prompt = "thumbnail for a learning content about 'Behavioural Skills',high res,attractive colors"
negative_prompt = ''

image_base = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    guidance_scale=9.0,
    num_inference_steps=25,
    ).images[0]

gc.collect()
torch.cuda.empty_cache()

In [ ]:
image_base

##  Using the refiner with the generated image.

In [ ]:
image_refiner = pipe_refiner(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=image_base,
    num_inference_steps=50,
    strength=0.3,
    ).images[0]

gc.collect()
torch.cuda.empty_cache()

In [ ]:
image_refiner

## SDXL Lightning

In [ ]:
base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_4step_unet.safetensors" # Use the correct ckpt for your step setting!

# Load model.
unet = UNet2DConditionModel.from_config(base, subfolder="unet").to("cuda", torch.float16)
unet.load_state_dict(load_file(hf_hub_download(repo, ckpt), device="cuda"))
pipe = StableDiffusionXLPipeline.from_pretrained(base, unet=unet, torch_dtype=torch.float16, variant="fp16").to("cuda")

In [ ]:
# Ensure sampler uses "trailing" timesteps.
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

In [ ]:
# Ensure using the same inference steps as the loaded model and CFG set to 0.
prompt = "thumbnail for a learning content about 'Behavioural Skills',high res,attractive colors"
image_lightning = pipe(prompt, num_inference_steps=4, guidance_scale=0, height=1024, width=1024).images[0]

In [ ]:
image_lightning